In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import pmdarima as pm

### Import

In [ ]:
filename = './data/ML_IGE_ENTRANTS_MODELE2.csv'

In [ ]:
df = pd.read_csv(filename,sep=';')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['SITE']=df['SITE'].str.strip()
df['SITE']=df['SITE'].astype('category')
df['SERVICE_ACTIVITE']=df['SERVICE_ACTIVITE'].astype('category')
df['DATEDATA']=pd.to_datetime(df['DATEDATA'])

In [ ]:
df.rename(columns={'SERVICE_ACTIVITE':'SERVICE','DATEDATA':'DATE'},inplace=True)

In [ ]:
df.head()

### Filter

In [ ]:
df['SITE'].unique()

In [ ]:
df['SERVICE'].unique()

In [ ]:
df_filter = df[(df['SITE']=='VELIZY') & (df['SERVICE']=='PRESTATION') & (df['DATE']<'2020-01-01') ]

In [ ]:
df_filter

### Create Aggregated Df with DatTime Index

In [ ]:
df2 = df_filter.groupby('DATE').sum()

In [ ]:
df2 = df2.asfreq(freq='B')

In [ ]:
df2.index

### Split in train test to compare with predictions

In [ ]:
from pmdarima.model_selection import train_test_split
train,test = train_test_split(df2, test_size=30)

In [ ]:
plt.rcParams['figure.figsize'] = [8, 5]
train.plot();

In [ ]:
train.info()

### Decompose

In [ ]:
# DECOMPOSE NOT WORKING !!!

# TO CHECK

In [ ]:
from pmdarima import utils

utils.autocorr_plot(train)
utils.plot_acf(train,zero=False)
utils.plot_pacf(train,zero=False)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
res = seasonal_decompose(train.TOTAL.interpolate(),period=5, model='additive')

# resplot = res.plot()

fig, (ax0,ax1,ax2,ax3) = plt.subplots(4,1, figsize=(20,8))
ax0.set_ylabel('TOTobAL')
ax1.set_ylabel('TREND')
ax2.set_ylabel('SEASONAL')
ax3.set_ylabel('RESIDUALS')


res.observed.plot(ax=ax0);
res.trend.plot(ax=ax1);
res.seasonal.plot(ax=ax2);
res.resid.plot(ax=ax3);

In [ ]:
res.observed[:10]

In [ ]:
plt.plot(res.observed[:10])

In [ ]:
plt.plot(train.TOTAL[:10])

### ACF / PACF

In [ ]:
# Fit a simple auto_arima model
modl = pm.auto_arima(train, error_action='ignore', trace=True,
                      suppress_warnings=True, maxiter=10,
                      seasonal=True, m=5)



In [ ]:
from sklearn.metrics import mean_squared_error
# Create predictions for the future, evaluate on test
preds, conf_int = modl.predict(n_periods=test.shape[0], return_conf_int=True)

# Print the error:
print("Test RMSE: %.3f" % np.sqrt(mean_squared_error(test, preds)))

In [ ]:
arima.seasonal_order

In [ ]:
import numpy as np
# #############################################################################
# Plot actual test vs. forecasts:
x = np.arange(test.shape[0])
plt.plot(x, test)
plt.plot(x, arima.predict(n_periods=test.shape[0]))
plt.title('Actual test samples vs. forecasts')
plt.show()

In [ ]:
utils.plot_acf(modl.resid)